In [ ]:
import numpy as np
from numpy.random import random
from numpy import vstack, hstack
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from Data.datasets import save_obj, load_obj

In [ ]:
df = pd.read_csv('Data/006.morph phase mapping_nonstd.csv')
df.index = list(df['index'])
df_score = df.filter(['score'], axis = 1)
df = df.drop(['index', 'score'], axis = 1)

# df_pool = load_obj('8R homogeneous concentration statespace (Pb2, morph, H2O and FAH constrained)')

# df = df_pool.filter(df_orig.index, axis = 'index')
# df_std = df_pool_std.filter(df_orig.index, axis = 'index')

In [ ]:
# df_pool['log(Pb)'] = list(np.log(df_pool['Pb']).replace([np.inf, -np.inf], 0))
# df_pool['log(morph)'] = list(np.log(df_pool['morph']).replace([np.inf, -np.inf], 0))
# df_pool['morph/Pb'] = df_pool['morph']/df_pool['Pb']
# df_pool['log(morph/Pb)'] = list(np.log(df_pool['morph/Pb']).replace([np.inf, -np.inf], 0))
# df_pool['log(H2O)'] = list(np.log(df_pool['H2O']).replace([np.inf, -np.inf], 0))
# df_pool['log(FAH)'] = list(np.log(df_pool['FAH']).replace([np.inf, -np.inf], 0))
# save_obj(df_pool, '8R homogeneous concentration statespace_new features (Pb2, morph, H2O and FAH constrained)')

# from sklearn.preprocessing import StandardScaler
# df_pool_std = StandardScaler().fit_transform(df_pool)
# df_pool_std = pd.DataFrame(columns = df_pool.columns, \
#                            index = df_pool.index, \
#                            data = df_pool_std)
# save_obj(df_pool_std, '8R homogeneous concentration statespace_new features_standardized (Pb2, morph, H2O and FAH constrained)')

### Visualize the points using [Morph]/[Pb] as a feature

In [ ]:
from itertools import combinations
clist = ['blue','green','red','orange']
slist = [150, 0, 40, 80]
alphalist = [0.5,0,0.8,0.8]

y_list = ['morph', 'Pb', 'FAH', 'H2O']

for i in list(combinations(y_list, 2)):
    fig = plt.figure(figsize = (6,6))
    ax = fig.add_subplot()
    
    ax.scatter(df[i[0]][df_score['score'] == 3],\
               df[i[1]][df_score['score'] == 3],\
               c = clist[2], s = slist[2], alpha = alphalist[2], linewidths = 0, label = 'Red phase')
    
    ax.scatter(df[i[0]][df_score['score'] == 4],\
               df[i[1]][df_score['score'] == 4],\
               facecolors='none', edgecolors=clist[3], linewidths = 1.5, s = slist[3], alpha = alphalist[3], label = 'Yellow phase')

#     ax.scatter(df[i[0]][df_score['score'] == 1,\
#                df[i[1]][df_score['score'] == 1],\
#                facecolors='none', edgecolors=clist[0], linewidths = 1, s = slist[0], alpha = alphalist[0], label = 'Clear solution')
    
    plt.xlabel(i[0])
    plt.ylabel(i[1])
    plt.legend()
    plt.savefig('Graphs_2/2D projection_'+i[0]+'_'+i[1]+'_5AL+KS.svg', format = "svg", transparent=True)

### Visualize the points in 3D space: [Morph]/[Pb], [FAH],and [H2O]

In [ ]:
coeff = np.array([-2.87693344,  1.07723441,  1.40760739])
intercept = np.array([-1.56718218])
xx, yy = np.meshgrid(range(12), range(16))
zz = (-coeff[0] * xx - coeff[1] * yy - intercept[0])/coeff[2]

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize = (6,6))
ax = fig.add_subplot(111, projection ='3d')

# plot the surface
ax = plt.figure().gca(projection='3d')
# ax.plot_surface(xx, yy, zz, alpha=0.2)

ax.scatter(df['log(morph/Pb)'][df_score['score'] == 3],\
           df['FAH'][df_score['score'] == 3],\
           df['H2O'][df_score['score'] == 3],\
           c = 'red', s = 20, alpha = 0.8, linewidths = 0, label = 'Red phase')
ax.scatter(df['log(morph/Pb)'][df_score['score'] == 4],\
           df['FAH'][df_score['score'] == 4],\
           df['H2O'][df_score['score'] == 4],\
           c = 'orange', s = 20, alpha = 0.8, linewidths = 0, label = 'yellow phase') # facecolors='none', edgecolors='blue', linewidths = 1.5, c = 'blue', s = 50, alpha = 0.2, label = 'Yellow phase'

#ax.set_xlim(0, 1)
#ax.set_ylim(0, 16)
#ax.set_zlim(0, 14)
ax.view_init(elev=21, azim=-64)
ax.set_xlabel('morph/Pb')
ax.set_ylabel('FAH')
ax.set_zlabel('H2O')
    
plt.legend()
plt.savefig('Graphs_2/3D projection_log(morph|Pb)_FAH, H2O for 5 AL.svg', format = "svg", transparent=True)

### Testing prediction accuracy just using [H2O] and [morph]/Pb

In [ ]:
prog_3vs4 = load_obj('index of class 3&4 for each AL iter_6KS')

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
clf = load_obj('RandomForestClassifier_best')

In [ ]:
df_std_prog.columns

In [ ]:
# calculate cross-validation score with original feature
accuracy_origin = []
for i in range(len(prog_3vs4)):
    df_std_prog = df_std.filter(prog_3vs4[i], axis = 0)
    df_score_prog = df_score.filter(prog_3vs4[i], axis = 0)
    score = cross_val_score(clf, np.array(df_std_prog.filter(['Pb', 'morph', 'DMSO', 'GBL', 'FAH', 'H2O'], axis = 1)), \
                            np.array(df_score_prog).ravel(), cv=cv)
    
    accuracy_origin.append([np.mean(score), np.std(score)])
    
accuracy_origin = pd.DataFrame(index = np.arange(len(prog_3vs4)), data = np.array(accuracy_origin), columns = ['mean', 'std'])
accuracy_origin.to_csv('cross_val score_original feature_RF.csv')

In [ ]:
# calculate cross-validation score with 'H2O', 'morph', 'Pb', 'FAH'
accuracy_1 = []
for i in range(len(prog_3vs4)):
    df_std_prog = df_std.filter(prog_3vs4[i], axis = 0)
    df_score_prog = df_score.filter(prog_3vs4[i], axis = 0)
    score = cross_val_score(clf, np.array(df_std_prog.filter(['morph','Pb','H2O','FAH'], axis = 1)), \
                            np.array(df_score_prog).ravel(), cv=cv)
    
    accuracy_1.append([np.mean(score), np.std(score)])
    
accuracy_1 = pd.DataFrame(index = np.arange(len(prog_3vs4)), data = np.array(accuracy_1), columns = ['mean', 'std'])
accuracy_1.to_csv('cross_val score_morph_Pb_H2O_FAH_RF.csv')

In [ ]:
# calculate cross-validation score with only morph/Pb and H2O as features
accuracy_2 = []
for i in range(len(prog_3vs4)):
    df_std_prog = df_std.filter(prog_3vs4[i], axis = 0)
    df_score_prog = df_score.filter(prog_3vs4[i], axis = 0)
    score = cross_val_score(GPC, np.array(df_std_prog.filter(['morph/Pb', 'H2O'], axis = 1)), \
                            np.array(df_score_prog).ravel(), cv=cv)
    
    accuracy_2.append([np.mean(score), np.std(score)])
    
accuracy_2 = pd.DataFrame(index = np.arange(len(prog_3vs4)), data = np.array(accuracy_2), columns = ['mean', 'std'])
accuracy_2.to_csv('cross_val score_morph|Pb_H2O_GPC.csv')

In [ ]:
# calculate cross-validation score with only morph/Pb as feature
accuracy_3 = []
for i in range(len(prog_3vs4)):
    df_std_prog = df_std.filter(prog_3vs4[i], axis = 0)
    df_score_prog = df_score.filter(prog_3vs4[i], axis = 0)
    score = cross_val_score(GPC, np.array(df_std_prog.filter(['morph/Pb'], axis = 1)), \
                            np.array(df_score_prog).ravel(), cv=cv)
    
    accuracy_3.append([np.mean(score), np.std(score)])
    
accuracy_3 = pd.DataFrame(index = np.arange(len(prog_3vs4)), data = np.array(accuracy_3), columns = ['mean', 'std'])
accuracy_3.to_csv('cross_val score_morph|Pb_GPC.csv')

In [ ]:
list(df_score.filter(prog_3vs4[6], axis = 0)['score']).count(3)

In [ ]:
list(df_score.filter(prog_3vs4[6], axis = 0)['score']).count(4)

### Analyzing feature importance through permutation of features

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm_imp = []

df_3vs4_6th = df_std.filter(prog_3vs4[6], axis = 0).filter(['Pb', 'morph', 'DMSO', 'GBL', 'FAH', 'H2O'], axis = 1)
df_3vs4_6th_score = df_score.filter(prog_3vs4[6], axis = 0)

for train_index, test_index in cv.split(df_3vs4_6th, df_3vs4_6th_score):
    x_train = np.array(df_3vs4_6th.iloc[train_index])
    y_train = np.array(df_3vs4_6th_score.iloc[train_index]).ravel()
    
    x_test = np.array(df_3vs4_6th.iloc[test_index])
    y_test = np.array(df_3vs4_6th_score.iloc[test_index]).ravel()
    
    GPC.fit(x_train, y_train)
    
    importance = PermutationImportance(GPC, random_state = 42).fit(x_test, y_test)
    perm_imp.append(list(importance.feature_importances_))

In [ ]:
perm_imp_mean = np.mean(perm_imp, axis = 0)
perm_imp_std = np.std(perm_imp, axis = 0)

df_feat_perm = pd.DataFrame(index = ['mean','std'], \
                            columns = df_3vs4_6th.columns, \
                            data = [perm_imp_mean, perm_imp_std])

df_feat_perm.to_csv('feature_importance_permutation_original_GPC.csv')